# Getting set up

## Setting up the notebook

In [45]:
### Installing dependencies
!pip install openai

!apt-get update
!apt-get install -y iverilog

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

## Use LLM to generate Verilog code from natural language description

### Example 1: binary_to_bcd

In [62]:
! mkdir -p binary_to_bcd
! cd binary_to_bcd && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/binary_to_bcd_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1078  100  1078    0     0   2302      0 --:--:-- --:--:-- --:--:--  2303


In [69]:
verilog_generation_prompt = '''
I am trying to create a Verilog model binary_to_bcd_converter for a binary to binary-coded-decimal converter. It must meet the following specifications:
	- Inputs:
		- Binary input (5-bits)
	- Outputs:
		- BCD (8-bits: 4-bits for the 10's place and 4-bits for the 1's place)

How would I write a design that meets these specifications?

module binary_to_bcd_converter (
    input  [4:0] bin_in,   // 5-bit binary input (0-31 decimal)
    output [7:0] bcd_out   // 8-bit BCD output: [7:4]=tens, [3:0]=ones
);

    // Internal registers for combinational logic
    reg [3:0] tens_reg;
    reg [3:0] ones_reg;

    // Combinational always block for BCD conversion
    always @(*) begin
        tens_reg = bin_in / 5'd10;  // Integer division (tens digit: 0-3)
        ones_reg = bin_in % 5'd10;  // Modulo (ones digit: 0-9)
    end

    // Assign BCD output (upper 4 bits = tens, lower 4 bits = ones)
    assign bcd_out = {tens_reg, ones_reg};

endmodule

binary_to_bcd_tb.v:8: error: port ``binary_input'' is not a port of uut.
binary_to_bcd_tb.v:8: error: port ``bcd_output'' is not a port of uut.
2 error(s) during elaboration.

match the signals with the test bench to avoid signal mismatch
'''

Now you can use the openai library to generate text from your prompt.

Before using LLMs for Verilog generation, you have to first get the api-key:

openai llms: https://platform.openai.com/api-keys
deepseek llms: https://platform.deepseek.com/sign_in
free llms provided by NVIDIA: https://build.nvidia.com/explore/discover

In [70]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "openrouter/sonoma-sky-alpha",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1700000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

Below is a corrected Verilog design for your `binary_to_bcd_converter` module. I'll explain the changes and rationale step by step, including how I addressed the testbench port mismatch error you encountered.

### Key Issues and Fixes
1. **Port Name Mismatch**:
   - Your module defines ports as `bin_in` (input) and `bcd_out` (output).
   - The testbench error indicates it's trying to connect to `binary_input` (input) and `bcd_output` (output) during instantiation of the unit-under-test (uut).
   - **Fix**: I've renamed the module's ports to `binary_input` and `bcd_output` to match what the testbench expects. This avoids elaboration errors without modifying the testbench (which you didn't provide). If you prefer to keep the original names, you'd need to update the testbench instantiation instead (e.g., change `.binary_input(uut_binary_input)` to `.bin_in(uut_binary_input)`).

2. **Overall Design**:
   - The module converts a 5-bit binary input (0–31 decimal) to an 8-bit BCD output, wher

Next, extract the generated verilog code from LLM response.

In [71]:
output_verilog_code = '''
module binary_to_bcd_converter (
    input  [4:0] binary_input,  // 5-bit binary input (0-31 decimal)
    output [7:0] bcd_output     // 8-bit BCD output: [7:4]=tens (0-3), [3:0]=ones (0-9)
);

    // Internal signals for combinational logic
    logic [3:0] tens;
    logic [3:0] ones;

    // Combinational always block for BCD conversion
    // Uses integer division and modulo (synthesizable for 5-bit input)
    always @(*) begin
        tens = binary_input / 5'd10;  // Tens digit: 0-3
        ones = binary_input % 5'd10;  // Ones digit: 0-9
    end

    // Concatenate to form 8-bit BCD output
    assign bcd_output = {tens, ones};

endmodule

'''
filename = "binary_to_bcd/binary_to_bcd.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

Use iverilog to verify the correctness of LLM generated verilog

In [72]:
!cd binary_to_bcd/ && iverilog -g2012 -o binary_to_bcd.vvp binary_to_bcd.v binary_to_bcd_tb.v && vvp binary_to_bcd.vvp

Testing Binary-to-BCD Converter...
VCD info: dumpfile my_design.vcd opened for output.
All test cases passed!


### Example 2: Sequence detector

In [73]:
! mkdir -p sequence_detector
! cd sequence_detector && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/sequence_detector_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2114  100  2114    0     0  10284      0 --:--:-- --:--:-- --:--:-- 10312


In [96]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a sequence detector. It must meet the following specifications:
	- Inputs:
		- Clock
		- Active-low reset
		- Data (3 bits)
	- Outputs:
		- Sequence found

While enabled, it should detect the following sequence of binary input values:
	- 0b001
	- 0b101
	- 0b110
	- 0b000
	- 0b110
	- 0b110
	- 0b011
	- 0b101

How would I write a design that meets these specifications?

Generated code :
module sequence_detector (
    input wire clk,
    input wire reset_n,
    input wire [2:0] data,
    output wire sequence_found
);

    reg [3:0] state;

    // State definitions (using numbers 0-8 for simplicity)
    localparam S0 = 4'd0;
    localparam S1 = 4'd1;
    localparam S2 = 4'd2;
    localparam S3 = 4'd3;
    localparam S4 = 4'd4;
    localparam S5 = 4'd5;
    localparam S6 = 4'd6;
    localparam S7 = 4'd7;
    localparam S8 = 4'd8;

    // Sequential logic for state transitions (synchronous reset)
    always @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            state <= S0;
        end else begin
            case (state)
                S0: begin
                    if (data == 3'b001) begin
                        state <= S1;
                    end else begin
                        state <= S0;
                    end
                end
                S1: begin
                    if (data == 3'b101) begin
                        state <= S2;
                    end else begin
                        state <= S0;
                    end
                end
                S2: begin
                    if (data == 3'b110) begin
                        state <= S3;
                    end else begin
                        state <= S0;
                    end
                end
                S3: begin
                    if (data == 3'b000) begin
                        state <= S4;
                    end else begin
                        state <= S0;
                    end
                end
                S4: begin
                    if (data == 3'b110) begin
                        state <= S5;
                    end else begin
                        state <= S0;
                    end
                end
                S5: begin
                    if (data == 3'b110) begin
                        state <= S6;
                    end else begin
                        state <= S0;
                    end
                end
                S6: begin
                    if (data == 3'b011) begin
                        state <= S7;
                    end else begin
                        state <= S0;
                    end
                end
                S7: begin
                    if (data == 3'b101) begin
                        state <= S8;
                    end else begin
                        state <= S0;
                    end
                end
                S8: begin
                    state <= S0;  // Reset to start after detection
                end
                default: begin
                    state <= S0;
                end
            endcase
        end
    end

    // Combinational logic for output (Moore machine: output depends on state)
    assign sequence_found = (state == S8);

endmodule

Tool Feedback:

Error: Cycle 8, Expected: 1, Got: 0

'''

In [97]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "openrouter/sonoma-sky-alpha",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1700000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

```verilog
module sequence_detector (
    input wire clk,
    input wire reset_n,
    input wire [2:0] data,
    output wire sequence_found
);

    reg [3:0] state;

    // State definitions (using numbers 0-7 for simplicity, no S8 needed)
    localparam S0 = 4'd0;
    localparam S1 = 4'd1;
    localparam S2 = 4'd2;
    localparam S3 = 4'd3;
    localparam S4 = 4'd4;
    localparam S5 = 4'd5;
    localparam S6 = 4'd6;
    localparam S7 = 4'd7;

    // Sequential logic for state transitions (synchronous reset)
    always @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            state <= S0;
        end else begin
            case (state)
                S0: begin
                    if (data == 3'b001) begin
                        state <= S1;
                    end else begin
                        state <= S0;
                    end
                end
                S1: begin
                    if (data == 3'b101) begin
                        state <= S2;

Next, extract the generated verilog code from LLM response.

In [98]:
output_verilog_code = '''
module sequence_detector (
    input wire clk,
    input wire reset_n,
    input wire [2:0] data,
    output wire sequence_found
);

    reg [3:0] state;

    // State definitions (using numbers 0-7 for simplicity, no S8 needed)
    localparam S0 = 4'd0;
    localparam S1 = 4'd1;
    localparam S2 = 4'd2;
    localparam S3 = 4'd3;
    localparam S4 = 4'd4;
    localparam S5 = 4'd5;
    localparam S6 = 4'd6;
    localparam S7 = 4'd7;

    // Sequential logic for state transitions (synchronous reset)
    always @(posedge clk or negedge reset_n) begin
        if (!reset_n) begin
            state <= S0;
        end else begin
            case (state)
                S0: begin
                    if (data == 3'b001) begin
                        state <= S1;
                    end else begin
                        state <= S0;
                    end
                end
                S1: begin
                    if (data == 3'b101) begin
                        state <= S2;
                    end else begin
                        state <= S0;
                    end
                end
                S2: begin
                    if (data == 3'b110) begin
                        state <= S3;
                    end else begin
                        state <= S0;
                    end
                end
                S3: begin
                    if (data == 3'b000) begin
                        state <= S4;
                    end else begin
                        state <= S0;
                    end
                end
                S4: begin
                    if (data == 3'b110) begin
                        state <= S5;
                    end else begin
                        state <= S0;
                    end
                end
                S5: begin
                    if (data == 3'b110) begin
                        state <= S6;
                    end else begin
                        state <= S0;
                    end
                end
                S6: begin
                    if (data == 3'b011) begin
                        state <= S7;
                    end else begin
                        state <= S0;
                    end
                end
                S7: begin
                    // On match, transition back to S0 (sequence complete)
                    // On mismatch, also back to S0 (no overlap assumed)
                    state <= S0;
                end
                default: begin
                    state <= S0;
                end
            endcase
        end
    end

    // Combinational logic for output (Mealy machine: output depends on state and input)
    // Assert output when in S7 and receiving the final sequence element
    assign sequence_found = (state == S7 && data == 3'b101);

endmodule
'''
filename = "sequence_detector/sequence_detector.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [99]:
!cd sequence_detector/ && iverilog -g2012 -o sequence_detector.vvp sequence_detector.v sequence_detector_tb.v && vvp sequence_detector.vvp

All test cases passed.


### Example 3: Shift Register

In [100]:
! mkdir -p shift_register
! cd shift_register && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/shift_register_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1367  100  1367    0     0   7015      0 --:--:-- --:--:-- --:--:--  7046


In [102]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a shift register. It must meet the following specifications:
	- Inputs:
		- Clock
		- Active-low reset
		- Data (1 bit)
		- Shift enable
	- Outputs:
		- Data (8 bits)

How would I write a design that meets these specifications?

'''

In [103]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "openrouter/sonoma-sky-alpha",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1700000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

Below is a complete Verilog design for an 8-bit serial-in, parallel-out (SIPO) shift register that meets your specifications. I'll explain the key design decisions and then provide the code.

### Design Explanation
- **Functionality**: 
  - The register shifts in 1 bit of serial data (`din`) from the least significant bit (LSB) position on each rising clock edge when the shift enable (`en`) is active (high).
  - This is a left-shifting register (i.e., `din` enters at bit 0, and the contents shift toward the most significant bit (MSB) at bit 7).
  - The parallel output is `q[7:0]`, where `q[7]` holds the oldest bit (shifted in 7 clocks ago) and `q[0]` holds the newest bit (just shifted in).
  - If you prefer right-shifting (e.g., `din` entering at the MSB), you can swap the concatenation in the shift logic to `{din, q[7:1]}`—let me know if you'd like that variant.
- **Reset**: Active-low asynchronous reset (`rst_n`). When `rst_n` is low, the entire register clears to 8'b0.
- **Enable**:

In [104]:
output_verilog_code = '''
module shift_register_8bit (
    input  wire clk,      // Clock input
    input  wire rst_n,    // Active-low reset input
    input  wire din,      // 1-bit serial data input
    input  wire en,       // Shift enable input (active high)
    output reg  [7:0] q   // 8-bit parallel data output
);

    // Asynchronous active-low reset and synchronous shift logic
    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            q <= 8'b0;  // Reset to all zeros
        end else if (en) begin
            q <= {q[6:0], din};  // Shift left: din enters at LSB (bit 0)
        end
        // If !en, q holds its current value (implicit)
    end

endmodule
'''
filename = "shift_register/shift_register.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [105]:
!cd sequence_detector/ && iverilog -o sequence_detector.vvp sequence_detector.v sequence_detector_tb.v && vvp sequence_detector.vvp

All test cases passed.




```
# This is formatted as code
```

### Example 4: Dice Roller

In [106]:
!mkdir -p dice_roller
!cd dice_roller && curl -O https://raw.githubusercontent.com/FCHXWH823/LLM4ChipDesign/fe806e8f8b7cb8442ce161f452d070cfcf953656/VerilogGenBenchmark/TestBench/dice_roller_tb.v

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3416  100  3416    0     0  21529      0 --:--:-- --:--:-- --:--:-- 21620


In [113]:
verilog_generation_prompt = '''
I am trying to create a Verilog model for a simulated dice roller. It must meet the following specifications:
    - Inputs:
        - Clock
        - Active-low reset
        - Die select (2-bits)
        - Roll
    - Outputs:
        - Rolled number (up to 8-bits)

The design should simulate rolling either a 4-sided, 6-sided, 8-sided, or 20-sided die, based on the input die select. It should roll when the roll input goes high and output the random number based on the number of sides of the selected die.

How would I write a design that meets these specifications?

Code Generated :
module dice_roller (
    input  wire        clk,        // Clock input
    input  wire        rst_n,      // Active-low reset
    input  wire [1:0]  die_sel,    // 2-bit die select: 00=4-sided, 01=6, 10=8, 11=20
    input  wire        roll,       // Roll trigger (high to roll)
    output reg  [7:0]  rolled_num  // Rolled number (1 to 20, upper bits 0)
);

    // Internal LFSR for pseudo-random generation (8 bits)
    reg [7:0] lfsr;

    // Detect rising edge of roll
    reg roll_prev;
    wire roll_rising_edge = roll & ~roll_prev;

    // LFSR feedback (simple polynomial for pseudo-randomness)
    wire feedback = lfsr[7] ^ lfsr[6] ^ lfsr[5] ^ lfsr[4];

    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            // Reset state
            lfsr        <= 8'h01;  // Non-zero seed to avoid all-zero state
            rolled_num  <= 8'h00;  // Default output
            roll_prev   <= 1'b0;   // No prior roll
        end else begin
            // Always update roll_prev for edge detection
            roll_prev <= roll;

            // Advance LFSR every clock cycle (for continuous randomness)
            lfsr <= {lfsr[6:0], feedback};

            // On rising edge of roll, compute and update output based on die_sel
            if (roll_rising_edge) begin
                case (die_sel)
                    2'b00: rolled_num <= (lfsr % 8'd4) + 8'd1;  // 4-sided: 1-4
                    2'b01: rolled_num <= (lfsr % 8'd6) + 8'd1;  // 6-sided: 1-6
                    2'b10: rolled_num <= (lfsr % 8'd8) + 8'd1;  // 8-sided: 1-8
                    2'b11: rolled_num <= (lfsr % 8'd20) + 8'd1; // 20-sided: 1-20
                    default: rolled_num <= 8'h00;              // Invalid sel: 0 (optional)
                endcase
            end
        end
    end

endmodule

Tool Feedback:
dice_roller_tb.v:10: error: port ``die_select'' is not a port of dut.
dice_roller_tb.v:10: error: port ``rolled_number'' is not a port of dut.
2 error(s) during elaboration.

'''

In [115]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "openrouter/sonoma-sky-alpha",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1700000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

Below is a complete Verilog design for the simulated dice roller that meets your specifications. I've made the following adjustments and improvements based on the provided code and tool feedback:

### Key Design Decisions and Fixes
- **Port Names**: Updated to match the testbench expectations from the tool feedback (`die_select` instead of `die_sel`, and `rolled_number` instead of `rolled_num`). This should resolve the elaboration errors in your testbench (`dice_roller_tb.v`).
- **Random Number Generation**: Retained the Linear Feedback Shift Register (LFSR) for pseudo-randomness, as it's a simple, synthesizable way to generate pseudo-random bits in hardware/simulation. The LFSR advances every clock cycle for continuous randomness.
- **Rolling Logic**: Detects the rising edge of the `roll` input (synchronous to `clk`). On a rising edge, it computes the rolled number based on `die_select` and latches it into the output. The output holds its value until the next roll.
- **Die Selection**

In [116]:
output_verilog_code = '''
module dice_roller (
    input  wire        clk,           // Clock input
    input  wire        rst_n,         // Active-low reset
    input  wire [1:0]  die_select,    // 2-bit die select: 00=4-sided, 01=6-sided, 10=8-sided, 11=20-sided
    input  wire        roll,          // Roll trigger (high to roll)
    output reg  [7:0]  rolled_number  // Rolled number (1 to 20, upper bits 0 for smaller dice)
);

    // Internal LFSR for pseudo-random generation (8 bits)
    reg [7:0] lfsr;

    // Detect rising edge of roll
    reg roll_prev;
    wire roll_rising_edge = roll & ~roll_prev;

    // LFSR feedback polynomial (simple primitive polynomial for good randomness: x^8 + x^6 + x^5 + x^4 + 1)
    wire feedback = lfsr[7] ^ lfsr[6] ^ lfsr[5] ^ lfsr[4];

    always @(posedge clk or negedge rst_n) begin
        if (!rst_n) begin
            // Reset state
            lfsr         <= 8'h01;       // Non-zero seed to avoid all-zero state
            rolled_number <= 8'h00;      // Default output (0)
            roll_prev    <= 1'b0;        // No prior roll
        end else begin
            // Always update previous roll for edge detection
            roll_prev <= roll;

            // Advance LFSR every clock cycle (for continuous pseudo-randomness)
            lfsr <= {lfsr[6:0], feedback};

            // On rising edge of roll, compute and latch output based on die_select
            if (roll_rising_edge) begin
                case (die_select)
                    2'b00: rolled_number <= (lfsr % 8'd4) + 8'd1;   // 4-sided: 1-4
                    2'b01: rolled_number <= (lfsr % 8'd6) + 8'd1;   // 6-sided: 1-6
                    2'b10: rolled_number <= (lfsr % 8'd8) + 8'd1;   // 8-sided: 1-8
                    2'b11: rolled_number <= (lfsr % 8'd20) + 8'd1;  // 20-sided: 1-20
                    default: rolled_number <= 8'h00;               // Invalid (shouldn't occur): 0
                endcase
            end
            // Output holds value otherwise
        end
    end

endmodule
'''
filename = "dice_roller/dice_roller.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [117]:
!cd dice_roller/ && iverilog -o dice_roller.vvp dice_roller.v dice_roller_tb.v && vvp dice_roller.vvp

VCD info: dumpfile my_design.vcd opened for output.
Results for die_select 00:
  Rolled           1:        254 times
  Rolled           2:        253 times
  Rolled           3:        250 times
  Rolled           4:        243 times
Results for die_select 01:
  Rolled           1:        168 times
  Rolled           2:        165 times
  Rolled           3:        163 times
  Rolled           4:        188 times
  Rolled           5:        164 times
  Rolled           6:        152 times
Results for die_select 10:
  Rolled           1:        129 times
  Rolled           2:        123 times
  Rolled           3:        127 times
  Rolled           4:        120 times
  Rolled           5:        127 times
  Rolled           6:        125 times
  Rolled           7:        125 times
  Rolled           8:        124 times
Results for die_select 11:
  Rolled           1:         58 times
  Rolled           2:         57 times
  Rolled           3:         54 times
  Rolled           4:

### Example 5: Token Bucket

In [160]:
!mkdir -p token_bucket/
!cd token_bucket/ && curl -O https://raw.githubusercontent.com/EMBEDDED-JUICE/TokenBucket/main/tokenbucket_tb.v


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4706  100  4706    0     0  28485      0 --:--:-- --:--:-- --:--:-- 28521


In [165]:
verilog_generation_prompt = '''
Write a Verilog module named token_bucket that implements a token bucket rate limiter.


Parameters:

DEN (default 16): denominator for fractional rate control (tokens per request).

RATE_NUM (default 3): number of tokens added per cycle.

BURST_MAX (default 8): maximum burst size, expressed in requests.

TOKEN_COST (default = DEN): tokens consumed per granted request.

Inputs:

clk: clock.

rst_n: active-low synchronous reset.

req_i: request input.

Outputs:

grant_o: asserted when a request is accepted and tokens are consumed.

ready_o: asserted whenever enough tokens are available for a request.

Functional Behavior:

Maintain an internal token counter (width at least 32 bits).

On every clock cycle, add RATE_NUM tokens, but do not exceed the maximum capacity (BURST_MAX * DEN).

If req_i is high and tokens ≥ TOKEN_COST:

Assert grant_o for one cycle.

Subtract TOKEN_COST tokens.

Otherwise, keep grant_o low.

ready_o is high whenever tokens ≥ TOKEN_COST.

On reset (rst_n = 0), initialize the token counter to full capacity (BURST_MAX * DEN) and clear grant_o.

'''

In [157]:
from openai import OpenAI
import os
client = OpenAI(base_url="https://openrouter.ai/api/v1",
    api_key = "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"
)

completion = client.chat.completions.create(
  model = "openrouter/sonoma-sky-alpha",
  messages=[{"role":"user","content":verilog_generation_prompt}],
  max_tokens=1700000, # limits the maximum number of tokens (words or pieces of words) that the model will generate in its response.
  stream=False
)

print(completion.choices[0].message.content)

```verilog
module token_bucket #(
    parameter DEN = 16,
    parameter RATE_NUM = 3,
    parameter BURST_MAX = 8,
    parameter TOKEN_COST = DEN
) (
    input wire clk,
    input wire rst_n,
    input wire req_i,
    output wire grant_o,
    output wire ready_o
);

    localparam [31:0] MAX_TOKENS = BURST_MAX * DEN;

    reg [31:0] tokens;
    reg grant_o_reg;

    assign grant_o = grant_o_reg;
    assign ready_o = (tokens >= TOKEN_COST);

    wire [31:0] added_tokens;
    assign added_tokens = (tokens + RATE_NUM > MAX_TOKENS) ? MAX_TOKENS : (tokens + RATE_NUM);

    always @(posedge clk) begin
        if (!rst_n) begin
            tokens <= MAX_TOKENS;
            grant_o_reg <= 1'b0;
        end else begin
            grant_o_reg <= 1'b0;
            if (req_i && (added_tokens >= TOKEN_COST)) begin
                grant_o_reg <= 1'b1;
                tokens <= added_tokens - TOKEN_COST;
            end else begin
                tokens <= added_tokens;
            end
        end
  

In [162]:
output_verilog_code = '''
module token_bucket #(
    parameter DEN = 16,
    parameter RATE_NUM = 3,
    parameter BURST_MAX = 8,
    parameter TOKEN_COST = DEN
) (
    input wire clk,
    input wire rst_n,
    input wire req_i,
    output wire grant_o,
    output wire ready_o
);

    localparam [31:0] MAX_TOKENS = BURST_MAX * DEN;

    reg [31:0] tokens;
    reg grant_o_reg;

    assign grant_o = grant_o_reg;
    assign ready_o = (tokens >= TOKEN_COST);

    wire [31:0] added_tokens;
    assign added_tokens = (tokens + RATE_NUM > MAX_TOKENS) ? MAX_TOKENS : (tokens + RATE_NUM);

    always @(posedge clk) begin
        if (!rst_n) begin
            tokens <= MAX_TOKENS;
            grant_o_reg <= 1'b0;
        end else begin
            grant_o_reg <= 1'b0;
            if (req_i && (added_tokens >= TOKEN_COST)) begin
                grant_o_reg <= 1'b1;
                tokens <= added_tokens - TOKEN_COST;
            end else begin
                tokens <= added_tokens;
            end
        end
    end

endmodule
'''
filename = "token_bucket/token_bucket.v"
# Write the extracted Verilog code to the file
with open(filename, "w") as f:
    f.write(output_verilog_code)

In [163]:
!cd token_bucket// && iverilog -o token_bucket.vvp token_bucket.v tokenbucket_tb.v && vvp token_bucket.vvp

VCD info: dumpfile token_bucket.vcd opened for output.
Requests total : 425
Grants (ref)   : 138
Grants (DUT)   : 138
Errors         : 0
RESULT: PASS (DUT == reference, shaped correctly)
Avg rate target ~ 3/16 = 0.187500 req/cycle
VCD written to token_bucket.vcd
